In [1]:
from tqdm import tqdm_notebook as notebook_tqdm

import torch
from transformers import AutoTokenizer

In [2]:
prompt = '''You are an agent filling forms on a website. You can issue these commands:
    - FILL////X////"Y": Fill the <input> or <textarea> element that has a relative xpath X with value Y
    - SELECT////X////"Y": Select the option Y in the <select> element that has a relative xpath X
Each command must be issued in a new line, with no preceding or trailing text values.
You are given a form to fill. The form's html is as follows:
<form _ngcontent-jwh-c36="" novalidate="" method="get" id="search-owner-form" class="form-horizontal ng-untouched ng-pristine ng-valid ng-submitted"><div _ngcontent-jwh-c36="" class="form-group"><div _ngcontent-jwh-c36="" id="lastNameGroup" class="control-group"><label _ngcontent-jwh-c36="" class="col-sm-2 control-label">Last name </label><div _ngcontent-jwh-c36="" class="col-sm-10"><input _ngcontent-jwh-c36="" size="30" maxlength="80" id="lastName" name="lastName" value="" class="form-control ng-untouched ng-pristine ng-valid"><span _ngcontent-jwh-c36="" class="help-inline"></span></div></div></div><div _ngcontent-jwh-c36="" class="form-group"><div _ngcontent-jwh-c36="" class="col-sm-offset-2 col-sm-10"><button _ngcontent-jwh-c36="" type="submit" class="btn btn-default">Find Owner</button></div></div></form>
The xpath in this format must be relative to the form element. Just give me the commands and nothing else.
YOUR COMMANDS:'''

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
def generate_text(prompt, max_length=100, temperature=0.7):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # Generate text
    output = model.generate(
        input_ids=input_ids,
        max_new_tokens=max_length,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
        # do_sample=True,
        # num_return_sequences=1
    )
    
    # Decode and return the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

## Bloom

In [4]:
from transformers import BloomForCausalLM

In [16]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m").to(device)

In [12]:
model = model.to(device)

In [13]:
generated_text = generate_text(prompt)

In [17]:
print(generated_text[len(prompt):])


- FILL////X////"Y": Fill the <input> or <textarea> element that has a relative xpath X with value Y
- SELECT////X////"Y": Select the option Y in the <select> element that has a relative xpath X
- FILL////X////"Y": Fill the <input> or <textarea> element that has a relative xpath X
- SELECT////X////"Y": Fill the <input> or <textarea> element that has a relative xpath X
- FILL////X


## BertGeneration

In [5]:
# !pip3 install transformers[sentencepiece]

In [6]:
from transformers import BertGenerationDecoder, BertGenerationConfig

In [8]:
tokenizer = AutoTokenizer.from_pretrained("google/bert_for_seq_generation_L-24_bbc_encoder")
config = BertGenerationConfig.from_pretrained("google/bert_for_seq_generation_L-24_bbc_encoder")
config.is_decoder = True
model = BertGenerationDecoder.from_pretrained(
    "google/bert_for_seq_generation_L-24_bbc_encoder", config=config
)

Some weights of BertGenerationDecoder were not initialized from the model checkpoint at google/bert_for_seq_generation_L-24_bbc_encoder and are newly initialized: ['lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
generated_text = generate_text(prompt)

IndexError: index -1 is out of bounds for dimension 1 with size 0

In [12]:
generated_text

'You are an agent filling forms on a website. You can issue these commands: - FILL////X////"Y": Fill the <input> or <textarea> element that has a relative xpath X with value Y - SELECT////X////"Y": Select the option Y in the <select> element that has a relative xpath XEach command must be issued in a new line, with no preceding or trailing text values.You are given a form to fill. The form\'s html is as follows:<form _ngcontent-jwh-c36="" novalidate="" method="get" id="search-owner-form" class="form-horizontal ng-untouched ng-pristine ng-valid ng-submitted"><div _ngcontent-jwh-c36="" class="form-group"><div _ngcontent-jwh-c36="" id="lastNameGroup" class="control-group"><label _ngcontent-jwh-c36="" class="col-sm-2 control-label">Last name </label><div _ngcontent-jwh-c36="" class="col-sm-10"><input _ngcontent-jwh-c36="" size="30" maxlength="80" id="lastName" name="lastName" value="" class="form-control ng-untouched ng-pristine ng-valid"><span _ngcontent-jwh-c36="" class="help-inline"></s